In [1]:
import pandas as pd
import brainlit
from brainlit.utils import *
import csv
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import plot_roc_curve
from sklearn import datasets, metrics, model_selection

from keras.models import Sequential
from keras.layers import Dense,  Activation
from keras.regularizers import L1L2
from keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder
from scipy import stats
import seaborn as sns

Using TensorFlow backend.


In [2]:
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
ngl_skel = NeuroglancerSession(url+"_segments", mip=1)
working_ids = []
for seg_id in range(300):
    try:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        working_ids.append(seg_id)
    except:
          pass

print (working_ids)

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.24it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.74it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.47it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.96it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 44.47it/s]

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


In [3]:
segment = ngl_skel.cv.skeleton.get(2)
ngl = NeuroglancerSession(url,mip=1)
voxel = ngl_skel._get_voxel(2, 4)
offset = np.array([15,15,15])
v_off = voxel + offset
voxel,v_off

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.96it/s]


(array([6432, 6170, 2226]), array([6447, 6185, 2241]))

In [18]:
output_feats = 'intensity_sel_all.csv'
int_verts = pd.DataFrame()
flag = 'w';
ids = working_ids
print (working_ids)

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


In [19]:
ids = working_ids
sel_ids=[]
for i in range(1,len(ids),5):
    sel_ids.append(ids[i])

In [17]:
df = pd.DataFrame()
with open(output_feats,flag) as f:
    for seg_id in sel_ids:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        for v_id,vertex in enumerate(segment.vertices):
            ngl = NeuroglancerSession(url,mip=1)
            try:
                if(v_id<50):
                    voxel = ngl_skel._get_voxel(seg_id, v_id)
                    offset = np.array([15,15,15])
                    v_off = voxel + offset
                    df = df.append(
                                {
                                    "Segment": int(seg_id),
                                    "Vertex": int(v_id),
                                    "Label": 1,
                                    "Features": voxel,
                                },
                                ignore_index=True,
                            )

                    df = df.append(
                                {
                                    "Segment": int(seg_id),
                                    "Vertex": int(v_id),
                                    "Label": 0,
                                    "Features": v_off,
                                },
                                ignore_index=True,
                            )

                    t0 = df['Features'][0]
                    t0 = t0.transpose()
                    t0 = np.append(t0,int(df['Label'][0]))
                    t1 = df['Features'][1]
                    t1 = t1.transpose()
                    t1 = np.append(t1,int(df['Label'][1]))
                    #print(t0)
                    writer = csv.writer(f)
                    writer.writerows([t0])
                    writer.writerows([t1])
                    #print(df)
                    df = pd.DataFrame()
                else:
                    break
            except:
                pass

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.46it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.02it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.90it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 36.49it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 31.99it/s]


...

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 53.43it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 56.12it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.35it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 43.09it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 49.95it/s]


In [28]:
df

""
